In [39]:
import copy
import random

class Q_learning_grid:
    def __init__(self,N :int,goal : str):
        if N<2:
            print("N should be greater than or equal to 2")
            exit(0)
        self.N = N
        self.goal = goal.lower()
        self.construct_Environment()
        self.construct_rewards_initial_Q()

    def construct_Environment(self):
        #n = int(input("Enter N:"))
        self.environment = {}
        k = 1
        for i in range(self.N):
            for j in range(self.N):
                self.environment[(i,j)] = f"s{k}"
                k+=1
        print(self.environment)
            
    def return_index(self,s : str):
        for k in self.environment:
            if self.environment[k] == s:
                return k

    def action_reward(self,action : int, i_p : int , j_p : int):#returns reward and next state index
        if action==0:
            i_n = i_p-1 #i_new
            if i_n<0:
                return [-100,i_p,j_p]#i_previous
            elif self.environment[(i_n,j_p)] == self.goal:
                return [100,i_n,j_p]
            else:
                return [0,i_n,j_p]
        elif action == 1:
            j_n = j_p+1
            if j_n>=self.N:
                return [-100,i_p,j_p]
            elif self.environment[(i_p,j_n)] == self.goal:
                return [100,i_p,j_n]
            else:
                return [0,i_p,j_n]
        elif action == 2:
            i_n = i_p+1
            if i_n>=self.N:
                return [-100,i_p,j_p]
            elif self.environment[(i_n,j_p)] == self.goal:
                return [100,i_n,j_p]
            else:
                return [0,i_n,j_p]
        elif action==3:
            j_n = j_p-1
            if j_n<0:
                return [-100,i_p,j_p]
            elif self.environment[(i_p,j_n)] == self.goal:
                return [100,i_p,j_n]
            else:
                return [0,i_p,j_n]  

    def construct_rewards_initial_Q(self):    
        self.rewards = {}
        k = 1
        i = 0
        j = 0 
        for i in range(self.N):
            for j in range(self.N):
                self.rewards[f"s{k}"] = [
                    self.action_reward(0,i,j)[0],#north
                    self.action_reward(1,i,j)[0],#east
                    self.action_reward(2,i,j)[0],#south
                    self.action_reward(3,i,j)[0]#west   
                ]
                k=k+1

        self.Q_table = copy.deepcopy(self.rewards)
        print(self.rewards)

    def make_learn(self,alpha=1.0,gamma=0.9,epochs = 20):
        self.alpha = alpha
        self.gamma = gamma
        start = "s1"
        if (start == self.goal):
            start = "s2"
        i = 0
        for i in range(20):
            state = start
            while state != self.goal:
                action = random.randint(0,3)
                rtplus1 = self.rewards[state][action]
                old_Q = self.Q_table[state][action]

                (a,b) = self.return_index(state)
                after_action = self.action_reward(action,a,b)
                if after_action[0] == -100:
                    next_maxQ = 0
                    self.Q_table[state][action] = rtplus1
                    continue
                next_state = self.environment[(after_action[1],after_action[2])]
                # print(f"{state}-->{next_state}")
                next_maxQ = max(self.Q_table[next_state])
                # print(next_maxQ)
                new_Q = ((1.0-self.alpha)*old_Q )+ (self.alpha*(rtplus1+(self.gamma*next_maxQ)))
                self.Q_table[state][action] = new_Q
                state = next_state
                # print(rewards)
            # print(self.Q_table)
        print(self.Q_table)
        
    def create_policy(self):
        policy = {}
        # print(Q_table)
        for state in self.Q_table:
            policy[state] = self.Q_table[state].index(max(self.Q_table[state]))
        print(policy)
        return policy
    
    def construct_path(self,start):
        start = start.lower()
        path = ""
        policy = self.create_policy()
        state = start 
        path = path+f"{state}"
        while state!=self.goal:
            action = policy[state]
            a,b = self.return_index(state)
            next_state_index = self.action_reward(action,a,b)
            next_state = self.environment[(next_state_index[1],next_state_index[2])]
            state = next_state
            path = path+f"-->{state}"
        return(path)

In [40]:
new_grid = Q_learning_grid(4,"s4")
new_grid.make_learn()
path = new_grid.construct_path("s16")
print(path)


{(0, 0): 's1', (0, 1): 's2', (0, 2): 's3', (0, 3): 's4', (1, 0): 's5', (1, 1): 's6', (1, 2): 's7', (1, 3): 's8', (2, 0): 's9', (2, 1): 's10', (2, 2): 's11', (2, 3): 's12', (3, 0): 's13', (3, 1): 's14', (3, 2): 's15', (3, 3): 's16'}
{'s1': [-100, 0, 0, -100], 's2': [-100, 0, 0, 0], 's3': [-100, 100, 0, 0], 's4': [-100, -100, 0, 0], 's5': [0, 0, 0, -100], 's6': [0, 0, 0, 0], 's7': [0, 0, 0, 0], 's8': [100, -100, 0, 0], 's9': [0, 0, 0, -100], 's10': [0, 0, 0, 0], 's11': [0, 0, 0, 0], 's12': [0, -100, 0, 0], 's13': [0, 0, -100, -100], 's14': [0, 0, -100, 0], 's15': [0, 0, -100, 0], 's16': [0, -100, -100, 0]}
{'s1': [-100, 81.0, 65.61000000000001, -100], 's2': [-100, 90.0, 72.9, 72.9], 's3': [-100, 100.0, 81.0, 81.0], 's4': [-100, -100, 0, 0], 's5': [72.9, 72.9, 59.049000000000014, -100], 's6': [81.0, 81.0, 65.61000000000001, 65.61000000000001], 's7': [90.0, 90.0, 72.9, 72.9], 's8': [100.0, -100, 81.0, 81.0], 's9': [65.61000000000001, 65.61000000000001, 53.144100000000016, -100], 's10': [72

In [28]:
# n = int(input("Enter N:"))
# # n= 3
# environment = {}
# k = 1
# for i in range(n):
#     for j in range(n):
#         environment[(i,j)] = f"s{k}"
#         k+=1

# print(environment)



In [29]:
# def return_index(s : str):
#     for k in environment:
#         if environment[k] == s:
#             return k

In [30]:
# goal = input("Enter Goal state:")
# start = "s1"
# # start = "s7"
# # goal = "s3"
# if (start == goal):
#     start = "s2"

# def action_reward(action : int, i_p : int , j_p : int):#returns reward and next state index
#     if action==0:
#         i_n = i_p-1 #i_new
#         if i_n<0:
#             return [-100,i_p,j_p]
#         elif environment[(i_n,j_p)] == goal:
#             return [100,i_n,j_p]
#         else:
#             return [0,i_n,j_p]
#     elif action == 1:
#         j_n = j_p+1
#         if j_n>=n:
#             return [-100,i_p,j_p]
#         elif environment[(i_p,j_n)] == goal:
#             return [100,i_p,j_n]
#         else:
#             return [0,i_p,j_n]
#     elif action == 2:
#         i_n = i_p+1
#         if i_n>=n:
#             return [-100,i_p,j_p]
#         elif environment[(i_n,j_p)] == goal:
#             return [100,i_n,j_p]
#         else:
#             return [0,i_n,j_p]
#     elif action==3:
#         j_n = j_p-1
#         if j_n<0:
#             return [-100,i_p,j_p]
#         elif environment[(i_p,j_n)] == goal:
#             return [100,i_p,j_n]
#         else:
#             return [0,i_p,j_n]
    
# rewards = {}
# k = 1
# i = 0
# j = 0 
# for i in range(n):
#     for j in range(n):
#         rewards[f"s{k}"] = [
#             action_reward(0,i,j)[0],#north
#             action_reward(1,i,j)[0],#east
#             action_reward(2,i,j)[0],#south
#             action_reward(3,i,j)[0]#west   
#         ]
#         k=k+1

# import copy
# Q_table = copy.deepcopy(rewards)
# print(rewards)

In [31]:
# import random
# alpha = 1.0
# gamma = 0.9

# i = 0
# for i in range(20):
#     state = start
#     while state != goal:
#         action = random.randint(0,3)
#         rtplus1 = rewards[state][action]
#         old_Q = Q_table[state][action]

#         (a,b) = return_index(state)
#         after_action = action_reward(action,a,b)
#         if after_action[0] == -100:
#             next_maxQ = 0
#             Q_table[state][action] = rtplus1
#             continue
#         next_state = environment[(after_action[1],after_action[2])]
#         # print(f"{state}-->{next_state}")
#         next_maxQ = max(Q_table[next_state])
#         # print(next_maxQ)
#         new_Q = ((1.0-alpha)*old_Q )+ (alpha*(rtplus1+(gamma*next_maxQ)))
#         Q_table[state][action] = new_Q
#         state = next_state
#         # print(rewards)
#     print(Q_table)


        

In [32]:
# def create_policy():
#     policy = {}
#     # print(Q_table)
#     for state in Q_table:
#         policy[state] = Q_table[state].index(max(Q_table[state]))
#     print(policy)
#     return policy
    
# def construct_path(start,goal):
#     path = ""
#     policy = create_policy()
#     state = start 
#     path = path+f"{state}"
#     while state!=goal:
#         action = policy[state]
#         a,b = return_index(state)
#         next_state_index = action_reward(action,a,b)
#         next_state = environment[(next_state_index[1],next_state_index[2])]
#         state = next_state
#         path = path+f"-->{state}"
#     return(path)

# start = input("Enter start state:")
# path = construct_path(start,goal)

# print(path)

    

exclude the snippets under this

In [ ]:

# environment = [
#     ["s1","s2","s3"],
#     ["s4","s5","s6"],
#     ["s7","s8","s9"]
# ]

# def get_index(s: str):
#     for i in range(len(environment)):
#         for j in range(len(environment)):
#             if environment[i][j]==s:
#                 return (i,j)

#stste : [north,east,south,west]
# rewards = {
#     "s1":[0,0,0,0],
#     "s2":[0,100,0,0],
#     "s3":[0,0,0,0],
# }
# "S7".lower()